In [1]:
#imports
import numpy as np
import pandas as pd
from hyperopt import hp
from hyperopt.pyll import scope
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models.hyperopt_model import HyperoptModel
from models.utils import CustomDataFrameMapper, get_features_facebook_qualitative
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr,  spearmanr,  kendalltau

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read the data
preprocessed_data = pd.read_pickle('../data/facebook_data_qualitative_scaled.pd')

# Create a dataframe
df = pd.DataFrame(preprocessed_data)

In [3]:
# Create two new dataframes, one with the training rows, one with the test rows
train, test=train_test_split(df, test_size=0.2, random_state=0)

In [4]:
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 9303
Number of observations in the test data: 2326


In [5]:
reaction_features, activity_features, user_features = get_features_facebook_qualitative()

In [7]:
    #some parameters
    DO_LOWESS = False
    hyperopt = True
    # Run Linear Regression with hyperopt optimization
    linear = HyperoptModel(train.copy(), test.copy(),'linear', cv=4, max_evals = 500)
    linear.raw_features = []
    linear.pipeline = Pipeline([
          ('prepare_features', FeatureUnion([
            ('user_features', user_features),
            ('activity_features', activity_features),
            ('reaction_features', reaction_features)
        ])),
        ('estimate', linear_model.LinearRegression())
    ])
    
    for transformer in linear.pipeline.named_steps['prepare_features'].transformer_list:
        linear.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
        
    linear.space = {
        'estimate__fit_intercept': hp.choice('estimate__fit_intercept', ['True', 'False']),
    }

    if hyperopt:
        linear.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        linear.pipeline.fit(X=linear.X_train, y=linear.y_train)
        linear.model = linear.pipeline
        linear.stats()
        linear.plot_feature_importance()
        linear.plot_predicted_vs_actual(do_lowess=DO_LOWESS)
        linear.qq_plot()

Performing parameters optimization...
Starting from scratch: new trials.
[1/500]	cv_eval_time=2.53 sec	RMSE=0.076477	R^2=0.714676
[2/500]	cv_eval_time=0.78 sec	RMSE=0.076574	R^2=0.714956
[3/500]	cv_eval_time=0.39 sec	RMSE=0.076637	R^2=0.714616
[4/500]	cv_eval_time=0.40 sec	RMSE=0.076494	R^2=0.714201
[5/500]	cv_eval_time=0.41 sec	RMSE=0.076524	R^2=0.714824
[6/500]	cv_eval_time=0.40 sec	RMSE=0.076544	R^2=0.714690
[7/500]	cv_eval_time=0.40 sec	RMSE=0.076569	R^2=0.714065
[8/500]	cv_eval_time=0.39 sec	RMSE=0.076534	R^2=0.714915
[9/500]	cv_eval_time=0.39 sec	RMSE=0.076582	R^2=0.715028
[10/500]	cv_eval_time=0.39 sec	RMSE=0.076513	R^2=0.714823
[11/500]	cv_eval_time=0.37 sec	RMSE=0.076562	R^2=0.714276
[12/500]	cv_eval_time=0.40 sec	RMSE=0.076461	R^2=0.714811
[13/500]	cv_eval_time=0.41 sec	RMSE=0.076480	R^2=0.715110
[14/500]	cv_eval_time=0.40 sec	RMSE=0.076450	R^2=0.714702


KeyboardInterrupt: 

In [10]:
    #some parameters
    DO_LOWESS = False
    hyperopt = False
    # Run Lasso Regression with hyperopt optimization
    lasso = HyperoptModel(train.copy(), test.copy(),'lasso', cv=4, max_evals = 500)
    lasso.raw_features = []
    lasso.pipeline = Pipeline([
          ('prepare_features', FeatureUnion([
           ('user_features', user_features),
           ('activity_features', activity_features),
           ('reaction_features', reaction_features)
        ])),
        ('estimate', linear_model.Lasso(**{'alpha': 0.0006577184991258585}))
    ])
    
    for transformer in lasso.pipeline.named_steps['prepare_features'].transformer_list:
        lasso.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
        
    lasso.space = {
        'estimate__alpha': hp.uniform('estimate__alpha', 0, 10),
    }

    if hyperopt:
        lasso.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        lasso.pipeline.fit(X=lasso.X_train, y=lasso.y_train)
        lasso.model = lasso.pipeline
        lasso.stats()
        lasso.plot_feature_importance()
        lasso.plot_predicted_vs_actual(do_lowess=DO_LOWESS)
        lasso.qq_plot()

Stats (train | test):
	R^2 score:		0.6937
					0.7083
	RMSE:			0.0793
					0.0775
	Mean error:		0.0623
					0.0610
	Pearson:		0.8369
					0.8466
	Spearman:		0.8159
					0.8316
	KendallTau:		0.6296
					0.6435

Plotting predicted vs. actual ...done

Plotting QQ ...done



In [11]:
    # Run CatBoostRegressor with hyperopt optimization
    DO_LOWESS = False
    hyperopt = True
    # Run CatBoostRegressor with hyperopt optimization
    cat = HyperoptModel(train.copy(), test.copy(),'cat', cv=3)
    cat.raw_features = []
    reaction_features, activity_features, user_features = get_features_facebook_qualitative()

    cat.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
           ('user_features', user_features),
            ('activity_features', activity_features),
            ('reaction_features', reaction_features)
        ])),
        ('estimate', CatBoostRegressor(**{'bagging_temperature': 0.7581941022055045,
                                          'iterations': 1300,
                                          'l2_leaf_reg': 3.3369699206218777,
                                          'learning_rate': 0.03533373267651768,
                                          'loss_function': 'RMSE',
                                          'random_seed': 0,
                                          'random_strength': 1,
                                          'thread_count': 4,
                                          'train_dir': 'outputs/cat'}))
    ])
    
    
    for transformer in cat.pipeline.named_steps['prepare_features'].transformer_list:
        cat.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
   
    cat.space = {
        'estimate__iterations': hp.choice('estimate__iterations', [1300]),
        'estimate__loss_function': hp.choice('estimate__loss_function', ['RMSE']),
        'estimate__train_dir': hp.choice('estimate__train_dir', ['outputs/cat']),
        'estimate__thread_count': hp.choice('estimate__thread_count', [4]),
        'estimate__used_ram_limit': hp.choice('estimate__used_ram_limit', [1024 * 1024 * 1024 * 4]),  # 4gb
        'estimate__random_seed': hp.choice('estimate__random_seed', [0]),

        'estimate__learning_rate': hp.loguniform('estimate__learning_rate', -5, 0),
        'estimate__random_strength': hp.choice('estimate__random_strength', [1, 20]),
        'estimate__l2_leaf_reg': hp.loguniform('estimate__l2_leaf_reg', 0, np.log(10)),
        'estimate__bagging_temperature': hp.uniform('estimate__bagging_temperature', 0, 1),
    }

    """
    find the best number of trees (following https://tech.yandex.com/catboost/doc/dg/concepts/parameter-tuning_trees-number-docpage/#parameter-tuning_trees-number)
    need to also substitute the CatBoostRegressor parameters with:
        ('estimate', CatBoostRegressor(iterations=10000, loss_function='RMSE', auto_stop_pval=1e-4, use_best_model=True, train_dir='outputs/cat_trees', verbose=True))
    """
    # num_trees_train, num_trees_eval = train_test_split(train, test_size=0.2, random_state=0)
    # X = num_trees_train[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # y = num_trees_train['score']
    #
    # eval_X = num_trees_eval[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # eval_X = cat.pipeline.named_steps['prepare_features'].fit_transform(eval_X)
    # eval_set = (eval_X, num_trees_eval['score'])
    #
    # cat.pipeline.fit(X=X, y=y, estimate__eval_set=eval_set)
    # cat.model = cat.pipeline
    # print(cat.model.named_steps['estimate'].get_params())
    # cat.model.named_steps['estimate'].save_model('tmp/cat.model'.encode('utf-8'))

    if hyperopt:
        cat.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        cat.pipeline.fit(X=cat.X_train, y=cat.y_train)
        cat.model = cat.pipeline
        cat.stats()
        cat.plot_predicted_vs_actual(do_lowess=DO_LOWESS)
        cat.plot_feature_importance()
        cat.qq_plot()

Performing parameters optimization...
Starting from scratch: new trials.


KeyboardInterrupt: 

In [13]:
    #parameters
    DO_LOWESS = False
    hyperopt = True
    # Run RandomForestRegressor with hyperopt optimization
    rf = HyperoptModel(train.copy(), test.copy(), 'rf', cv=3)
    rf.raw_features = []

    rf.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_features', user_features),
            ('activity_features', activity_features),
            ('reaction_features', reaction_features)
        ])),
        ('estimate', RandomForestRegressor(**{'max_features': 0.5954111977396234,
                                              'min_samples_leaf': 1,
                                              'n_estimators': 388,
                                              'oob_score': True,
                                              'random_state': 0}))
    ])
    
    
    for transformer in rf.pipeline.named_steps['prepare_features'].transformer_list:
        rf.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
     # for transformer in rf.pipeline.named_steps['prepare_features'].transformer_list:
      #    rf.raw_features += [t if isinstance(t, basestring) else t[0] for t, _, _ in transformer[1].features]
    #print(rf.get_params().keys())
    rf.space = {
        'estimate__random_state': hp.choice('estimate__random_state', [0]),
        'estimate__oob_score': hp.choice('estimate__oob_score', [True]),

        'estimate__max_features': hp.uniform('estimate__max_features', 0, 1.),
        'estimate__n_estimators': hp.choice('estimate__n_estimators', range(1, 3000 + 1)),
        'estimate__min_samples_leaf': hp.choice('estimate__min_samples_leaf', range(1, 100 + 1)),
        #'estimate__scale': hp.choice('estimate__scale', [0, 1.]),
        #'estimate__normalize': hp.choice('estimate__normalize', [0, 1.]),
    }

    if hyperopt:
        rf.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        rf.pipeline.fit(X=rf.X_train, y=rf.y_train)
        rf.model = rf.pipeline
        rf.stats()
        rf.plot_predicted_vs_actual()
        rf.plot_feature_importance()

Performing parameters optimization...
Starting from scratch: new trials.


KeyboardInterrupt: 

In [19]:
    #some parameters
    DO_LOWESS = False
    hyperopt = True
    # Run MLPRegressor with hyperopt optimization
    nn = HyperoptModel(train.copy(), test.copy(),'nn', cv=3, max_evals=2)
    nn.raw_features = []

    nn.pipeline = Pipeline([
            ('prepare_features', FeatureUnion([
           ('user_features', user_features),
            ('activity_features', activity_features),
            ('reaction_features', reaction_features)
        ])),
        ('estimate', MLPRegressor(**{'alpha': 0.00189292968732814,
                                     'activation': 'logistic',
                                     'hidden_layer_sizes': (88,),
                                     'solver': 'adam'    }))
    ])
    
    for transformer in nn.pipeline.named_steps['prepare_features'].transformer_list:
        nn.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
   

    nn.space = {
         'estimate__alpha' : hp.uniform('estimate__alpha', 0.00001, 1),
         'estimate__activation' : hp.choice('estimate__activation', ['logistic']), # 'identity', 'logistic', 'tanh', 'relu'
         #'estimate__learning_rate' : hp.choice('estimate__learning_rate', ['constant', 'invscaling', 'adaptive']),
         'estimate__hidden_layer_sizes' : (scope.int(hp.uniform('estimate__first_layer', 1, 100)),scope.int(hp.uniform('estimate__second_layer', 1, 100))) ,
         'estimate__solver' : hp.choice('estimate__solver', ['adam']), #'lbfgs', 'sgd',
         #'estimate__max_iter' : scope.int(hp.uniform('estimate__max_iter', 500, 1000))
    }

    if hyperopt:
        nn.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        nn.pipeline.fit(X=nn.X_train, y=nn.y_train)
        nn.model = nn.pipeline
        nn.stats()
        nn.plot_predicted_vs_actual(do_lowess=DO_LOWESS)
        nn.plot_feature_importance()
        nn.qq_plot()

Performing parameters optimization...
Starting from scratch: new trials.
[1/2]	cv_eval_time=7.37 sec	RMSE=0.143400	R^2=-0.002252
[2/2]	cv_eval_time=12.22 sec	RMSE=0.146940	R^2=-0.090670
 elapsed time: 19.7 s

Stats (train | test):
	R^2 score:		-0.0029
					-0.0030
	RMSE:			0.1436
					0.1438
	Mean error:		0.1130
					0.1140
	Pearson:		0.5171
					0.5082
	Spearman:		0.4844
					0.4777
	KendallTau:		0.3362
					0.3297

Best parameters set:
{'estimate__activation': 'logistic',
 'estimate__alpha': 0.9205859728209516,
 'estimate__hidden_layer_sizes': (19, 19),
 'estimate__solver': 'adam'}

Plotting predicted vs. actual ...done

Plotting QQ ...done

